In [4]:
"""Hello."""
import sys
import numpy
import random
import operator
import math
import itertools

from collections import defaultdict
from pprint import pprint

#filename = "test.in"
filename = "busy_day.in"
#filename = "redundancy.in"
#filename = "mother_of_all_warehouses.in"
debug = False

def scale(order):
    """Total weight of `order` format e.g. [0,2,1,0,0] - amount of goods."""
    return sum(map(lambda prodtype: weights[prodtype], to_types(order)))


def to_types(order):
    """`order` format to `types` e.g. [0,2,1,0,0] -> [1, 1, 2]."""
    for prodtype, amount in enumerate(order):
        for _ in range(amount):
            yield prodtype


def from_types_init(P):
    """`types` format to `order` e.g. [1, 1, 2] -> [0,2,1,0,0]."""
    def from_types(prodtypes):
        order = numpy.zeros(P, dtype='i4')
        for prodtype in prodtypes:
            order[prodtype] += 1
        return order
    return from_types


class Place:
    """Base class for delivering goods - to warehouse or customer."""

    def __init__(self, uid, pos):
        self.uid = uid
        self.pos = pos

    def distance(self, other):
        return math.sqrt((self.pos[0] - other.pos[0])**2 + (self.pos[1] - other.pos[1])**2)

    def __str__(self):
        return str(self.pos)


class Customer(Place):
    def __init__(self, uid, pos, P, L, order):
        super(Customer, self).__init__(uid, pos)
        self.order = from_types(order)

    def __str__(self):
        return "C#{:d} at {} asking {!s}".format(self.uid, self.pos, list(to_types(self.order)))


class Warehouse(Place):
    def __init__(self, uid, pos, wares):
        super(Warehouse, self).__init__(uid, pos)
        # how many wares it actually has
        self.wares = wares
        # assigned customers
        self.customers = []
        # how many wares does it need
        self.supply = wares[:]

    def __str__(self):
        return "W#{:d} at {} with {}".format(self.uid, self.pos, self.supply)


class Order:
    def __init__(self, pos, wares):
        super(Order, self).__init__(pos)
        self.wares = wares


class Drone:
    def __init__(self, uid, capacity, start, P):
        self.uid = uid
        self.capacity = capacity
        self.cargo = numpy.zeros(P, dtype='i4')
        self.at = start
        self.time = 0  # time of completion of currently scheduled operations

    def carry(self):
        return scale(self.cargo)

    def load(self, prodtype):
        if weights[prodtype] + self.carry() > self.capacity:
            return False
        self.cargo[prodtype] += 1
        return True

    def send(self, pfrom, pto, plan):
        self.time += round(self.at.distance(pfrom))
        for prodtype, amount in enumerate(self.cargo):
            if amount == 0:
                continue
            plan.append((self.uid, "L", pfrom.uid, prodtype, amount))
        for prodtype, amount in enumerate(self.cargo):
            if amount == 0:
                continue
            if isinstance(pto, Warehouse):
                plan.append((self.uid, "U", pto.uid, prodtype, amount))
            if isinstance(pto, Customer):
                plan.append((self.uid, "D", pto.uid, prodtype, amount))
        self.at = pto
        self.time += round(pfrom.distance(pto))
        self.cargo = numpy.zeros(P, dtype='i4')

    def wait(self, until, plan):
        if self.time < until:
            plan.append((self.uid, "W", int(until - self.time)))
            self.time = until

def transport(pfrom, pto, order, drones, plan):
    """
    Returns:
    actions per round per drone,
    remaining order,
    unused drones

    1. We harness all drones (ideal state)
    2. We have more drones -> return unused drones
    3. We are missing drones -> return remaining order
    Sort drones by the distance from warehouse (pfrom).
    Estimate amount of drones for the transporatation.
    Move all used drones to pfrom.
    Load the wares.
    Drones will move to `pto`.
    Unload the wares.
    """
    if scale(order) == 0:
        return
    drone = next(drones)
    for prodtype in to_types(order):
        if not drone.load(prodtype):
            drone.send(pfrom, pto, plan)
        else:
            drone = next(drones)
            drone.load(prodtype)
    if drone.carry() > 0:
        drone.send(pfrom, pto, plan)


In [5]:

# kick-ass reading submission from a file

f = open(filename, "r")
line = next(f)
rows, cols, D, time, load = map(int, line.split())

P = int(next(f))
weights = [float(l) for l in next(f).strip().split()]
from_types = from_types_init(P)

W = range(int(next(f)))
warehouses = [
    Warehouse(w, tuple(map(int, next(f).strip().split())), list(map(int, next(f).strip().split()))) for w in W
]

C = range(int(next(f)))
customers = [
    Customer(c, 
             tuple(map(int, next(f).strip().split())),  # pos
             P,
             int(next(f).strip()),
             list(map(int, next(f).strip().split())))
    for c in C
]

_drones = [Drone(d, load, warehouses[0], P) for d in range(D)]
drones = itertools.cycle(_drones)

f.close()

if debug:
    print(list(map(str, warehouses[:5])))
    print(list(map(str, customers[:5])))

In [6]:
plan = []

# assign customers to their nearest warehouse
for customer in customers:
    distances = [(warehouse.uid, customer.distance(warehouse)) for warehouse in warehouses]
    warid, distance = min(distances, key=operator.itemgetter(1))
    warehouses[warid].customers.append(customer)
    if debug:
        print("W#{0.uid} at {0.pos} got C#{1.uid} at {1.pos}".format(warehouses[warid], customer))

# plan transportation of goods between warehouses to fulfil demand of customers
for warehouse in warehouses:
    for customer in warehouse.customers:
        for prodtype, o in enumerate(customer.order):
            warehouse.supply[prodtype] -= o

if debug:
    for w in warehouses:
        print("W#{0.uid} has {0.wares} and needs {0.supply}".format(w))

# transport the planned goods
transports = dict()
for w1 in W:
    for w2 in W:
        transports[(w1, w2)] = numpy.zeros(P, dtype='i4')

for warehouse in warehouses:
    for prodtype, _ in enumerate(warehouse.supply):
        if warehouse.supply[prodtype] >= 0:
            continue
        for supplier in warehouses:
            if supplier.uid == warehouse.uid:
                continue
            if supplier.supply[prodtype] > 0:
                delta = min(warehouse.supply[prodtype] * -1, supplier.supply[prodtype])
                transports[(supplier.uid, warehouse.uid)][prodtype] += delta
                warehouse.supply[prodtype] += delta
                supplier.supply[prodtype] -= delta
                if warehouse.supply[prodtype] >= 0:
                    break

for (pfrom, pto), order in transports.items():
    transport(warehouses[pfrom], warehouses[pto], order, drones, plan)

# wait for all LOAD/UNLOAD to finish
max_time = max(_drones, key=operator.attrgetter('time')).time
for drone in _drones:
    drone.wait(max_time, plan)

# distribute the goods to customers
for warehouse in warehouses:
    for customer in warehouse.customers:
        transport(warehouse, customer, customer.order, drones, plan)

with open(filename+".result", "wt") as fo:
    print(len(plan), file=fo)
    for step in plan:
        print(*step, file=fo)
print("Done")

Done
